In [1]:
from selenium import webdriver
from selenium.webdriver.remote.errorhandler import NoSuchElementException
from datetime import datetime as dt
import yaml
import time

In [2]:
def athena_driver(username, password, headless=True, download_dir=None):
    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    options.add_argument('window-size=1680x900')
    options.headless = headless
    if download_dir:
        options.add_experimental_option('prefs', {'download.default_directory': download_dir})
    driver = webdriver.Chrome(executable_path='chromedriver.exe', options=options)
    driver.set_window_position(0,0)
    driver.set_window_size(1200, 1375)
    driver.set_page_load_timeout(15)
    
    driver.get("https://athena.ohdsi.org/")
    driver.find_element_by_xpath('/html/body/div/div/div[4]/div[2]/div/div[2]/div[2]/button').click()
    driver.find_element_by_xpath('/html/body/div/div/header/nav/div[3]/a').click()
    driver.find_element_by_xpath('/html/body/div/div/div[1]/div/div[2]/div/button').click()

    main_window = driver.current_window_handle
    for handle in driver.window_handles:
        if handle != main_window:
            popup = handle
            driver.switch_to.window(popup)

    driver.find_element_by_xpath('//*[@id="username"]').send_keys(username)
    driver.find_element_by_xpath('//*[@id="password"]').send_keys(password)
    driver.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/div/form/section[3]/input[4]').click()
    driver.switch_to.window(driver.window_handles[0])

    return driver 

In [3]:
def request_new_vocabulary_file(driver):
    driver.get('https://athena.ohdsi.org/search-terms/start')
    driver.find_element_by_xpath('/html/body/div/div/header/nav/div[3]/div/a/div[2]').click()
    driver.find_element_by_xpath('//*[@id="app"]/div/header/nav/div[3]/div/div/div/a[1]').click()
    driver.find_element_by_xpath('//*[@id="app"]/div/header/nav/div[2]/a').click()
    
    # double click on select all to make sure no vocab is selected 
    driver.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[2]/table/thead/tr/th[1]/label').click()
    driver.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[2]/table/thead/tr/th[1]/label').click()

    # Select vocabularies
    checkboxes = driver.find_elements_by_css_selector("#app > div > div.at-vocabs > div.at-vocabularies > table > tbody > tr")
    with open('omop_vocabularies.yml') as f:
        vocabs = yaml.safe_load(f)
    
    for checkbox in checkboxes:
        cls = checkbox.find_element_by_class_name('at-vocabularies__code-td')
        if cls.text in vocabs:
            checkbox.click()
            print(f'Vocabulary {cls.text} was selected')
    
    datestamp = dt.today().strftime('%m_%d_%Y')
    driver.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[1]/button').click()
    driver.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[3]/div[2]/div/div[2]/form/div/div[1]/input').send_keys(f'vocabulary_5x_{datestamp}')
    
    # Request vocab file
    driver.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[3]/div[2]/div/div[2]/form/div/div[2]/button[1]').click()

    #return to main page
    driver.get('https://athena.ohdsi.org/search-terms/start')

In [4]:
def download_vocabulary_file(driver, get_last=True, vocabulary_name=None, archive=True, restore=True):
    """Download most recent vocabulary file."""
    driver.get('https://athena.ohdsi.org/search-terms/start')
    driver.find_element_by_xpath('/html/body/div/div/header/nav/div[3]/div/a/div[2]').click()
    driver.find_element_by_xpath('//*[@id="app"]/div/header/nav/div[3]/div/div/div/a[1]').click()
    athena.find_element_by_xpath('//*[@id="app"]/div/header/nav/div[3]/div').click()
    buttons = driver.find_elements_by_class_name("react-sanfona-item")
    if get_last: 
        button = buttons[0]
        tool = button.find_element_by_class_name('ac-toolbar')
        status = tool.text.split()[-1]
        print(status)
        if status == 'DOWNLOADSHAREARCHIVE':
            print(tool.text)
            tool.click()
        elif status == 'ARCHIVEDRESTORE':
            if restore:
                button.click()
                print('Restoring vocabulary, please wait.')
                while status == 'PENDING':
                    tool = button.find_element_by_class_name('ac-toolbar')
                    status = tool.text.split()[-1]
                    if status == 'DOWNLOADSHAREARCHIVE':
                        tool.click()
                        print("Download started")
            else:
                return 'Vocabulary file is archived. Set restore=True to restore and download.'

        if archive:
            driver.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[2]/div[1]/div[1]/ul/li/div/button[3]').click()

    driver.get('https://athena.ohdsi.org/search-terms/start')

In [2]:
# Login to Athena
#TODO - get an account that everyone can access.
USERNAME = "yankuic@ufl.edu"
PASSWORD = "7DSrMx7JNTHaXEM"
path = r'Z:\OMOP_CDM'
athena = athena_driver(USERNAME, PASSWORD, headless=False, download_dir=path)
# request_new_vocabulary_file(athena)

In [39]:
# download_vocabulary_file(athena)
# athena.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[2]/div[2]/div[1]/ul/li/div/button').click()
athena

DOWNLOADSHAREARCHIVE
keyboard_arrow_right
VOCABULARY_5X_04_13_2021
04:11 PM | 13-APR-2021
V5.0 02-APR-21
DOWNLOADSHAREARCHIVE


In [6]:
# athena.switch_to.window(athena.window_handles[0])
athena.get('https://athena.ohdsi.org/search-terms/start')
athena.find_element_by_xpath('/html/body/div/div/header/nav/div[3]/div/a/div[2]').click()
athena.find_element_by_xpath('//*[@id="app"]/div/header/nav/div[3]/div/div/div/a[1]').click()
athena.find_element_by_xpath('//*[@id="app"]/div/header/nav/div[3]/div').click()
buttons = athena.find_elements_by_class_name("react-sanfona-item")
last = True
archive = False
if last:
    button = buttons[0]
    tool = button.find_element_by_class_name('ac-toolbar')
    status = tool.text.split()[-1]
    if status == 'DOWNLOADSHAREARCHIVE': 
        athena.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[2]/div[1]/div[1]/ul/li/div/button[1]').click()
        if archive:
            time.sleep(30)
            athena.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[2]/div[1]/div[1]/ul/li/div/button[3]').click()
    elif status == 'ARCHIVEDRESTORE':
        athena.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[2]/div[1]/div[1]/ul/li/div/button').click()
        time.sleep(2)
        tool = button.find_element_by_class_name('ac-toolbar')
        status = tool.text.split()[-1]
        while status == 'PENDING':
            print('Restoring vocabulary, please wait.', end='\r')
            time.sleep(60)
            tool = button.find_element_by_class_name('ac-toolbar')
            status = tool.text.split()[-1]
            if status == 'DOWNLOADSHAREARCHIVE':
                time.sleep(30)
                athena.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[2]/div[1]/div[1]/ul/li/div/button[1]').click()
                print("Download started")
                if archive:
                    time.sleep(30)
                    athena.find_element_by_xpath('//*[@id="app"]/div/div[1]/div[2]/div[1]/div[1]/ul/li/div/button[3]').click()                
    elif status == 'PENDING':
        print('Restoring vocabulary, try again later.')

In [8]:
import glob, os
csv_list = glob.glob(path + '/*.zip')
last_zip = max(csv_list, key=os.path.getctime)

In [12]:
import zipfile
with zipfile.ZipFile(last_zip) as z:
    z.extractall(path + '/Vocabulary_v5x')
    print('Vocabulary files were extracted.')

Vocabulary files were extracted.


In [36]:
import datetime
import time
import math
from pydma.utils import timeit_context
import pandas as pd
import numpy as np
import numpy.ma as ma
from turbodbc import connect
from turbodbc.exceptions import DatabaseError, InterfaceError
from pydma.databases import OneFLDb

database='DWS_OMOP'
omop = OneFLDb('edw', database=database)
today = datetime.datetime.today()
date_flag = today.strftime('%Y%m%d')
schema = 'testing'

t_connection = connect(driver='{SQL Server}', server='edw.shands.ufl.edu', 
              database=database, trusted_connection='yes')

tables = ['drug_strength']#['concept', 'concept_ancestor','concept_class', 'concept_relationship', 'concept_synonym'
#           'domain', 'drug_strength', 'relationship', 'vocabulary']
df_debug = ''

for table in tables:
    # Truncate table
    cursor = t_connection.cursor()
    
    sql = """\
        truncate table {}.{}
        """.format(schema, table)
    
    # Use raw_connection to access data types.
    raw_connection = omop.engine.raw_connection()
    omop_cursor = raw_connection.cursor()
    omop_cursor.execute(sql)
    omop_cursor.commit()
          
    # Retrieve sql data types 
    omop_cursor.execute(f'select top 1 * from {schema}.{table}')
    dtypes = {t[0]:t[1] for t in omop_cursor.description}
#     nullable = [t[0] for t in omop_cursor.description if t[-1]]
    omop_cursor.close()
    raw_connection.close()
    
    # Use turbodbc supported data types
    for t in dtypes.keys():
        if dtypes[t] is int:
            dtypes[t] = np.int64
    
    if table == 'drug_strength':
        chunks = pd.read_csv(r'z:\OMOP_CDM\Vocabulary_v5x\{}.csv'.format(table.upper()), #keep_default_na=False, 
                             chunksize=1000000, sep='\t', dtype=str, low_memory=False)

    else:
        chunks = pd.read_csv(r'z:\OMOP_CDM\Vocabulary_v5x\{}.csv'.format(table.upper()), keep_default_na=False,
                         chunksize=1000000, dtype=dtypes, sep='\t')
    
    insert_query = """
            SET ANSI_WARNINGS OFF
            
            INSERT INTO {0}.{1} ({2})
            VALUES ({3})
            
            SET ANSI_WARNINGS ON
        """
    
    remove_999 = """
            update {0}.{1}
            set {2} = NULL
            where {2} = -999
    """
    
    count = 0
    rows_proc = 0
    with timeit_context(f'Processing table {table}'):
        print(f'Loading table {table} into schema {schema}. Please wait.')
        for chunk in chunks:
            
            if table == 'drug_strength':
                
                for col in chunk.columns:
                    data_type = dtypes[col]
                    if data_type is np.int64:
                        chunk[col] = chunk[col].fillna(-999).astype(data_type)
                    elif data_type is str:
                        chunk[col] = chunk[col].fillna('').astype(data_type)
                
            # Replace all NaN for None - turbodbc does not support NaN
            chunk = chunk.where((pd.notnull(chunk)), None)

            if count == 0:
                columns = ','.join(chunk.columns)
                n_cols = len(chunk.columns)
                placehl = ','.join(['?']*n_cols)
            count=+1

            insert_query = insert_query.format(schema, table, columns, placehl)

            try:
                cursor.executemanycolumns(insert_query, [np.ascontiguousarray(chunk[col].values) for col in chunk.columns])
                rows_proc = rows_proc + chunk.shape[0]
                t_connection.commit()
                                
            except (DatabaseError, InterfaceError, ValueError) as e:
                df_debug = chunk
                t_connection.rollback()
                t_connection.close()
                raise e
    
    if table == 'drug_strength':
        
        for col in chunk.columns:
            data_type = dtypes[col]
            if data_type is np.int64:
                print(f'Cleaning up column {col}')
                cursor.execute(remove_999.format(schema, table, col))
                t_connection.commit()

    cursor.close()            
    print(f'Import complete: {rows_proc} rows processed')

t_connection.close()

Loading table drug_strength into schema testing. Please wait.
Processing table drug_strength finished in 00 h 02 m 05 s
drug_concept_id
ingredient_concept_id
amount_unit_concept_id
numerator_unit_concept_id
denominator_unit_concept_id
box_size
2829451 rows processed
